<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2024Project/blob/Narzdzie-do-wizualizacji-na-mapach/mapy_datashader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datashader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 62.6 MB/s eta 0:00:00


In [2]:
!pip install holoviews hvplot colorcet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 29.7 MB/s eta 0:00:00


In [3]:
!pip install geoviews

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.2/511.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 47.6 MB/s eta 0:00:00


In [8]:
import datashader as ds
import datashader.transfer_functions as tf
import numpy as np
import colorcet as cc
import holoviews as hv
# from holoviews.element.tiles import EsriImagery
from holoviews.operation.datashader import datashade
import pandas as pd
import geoviews as gv
# import hvplot.pandas
import geoviews.tile_sources as gts
from holoviews import opts
import matplotlib as mpl
import matplotlib.pyplot as plt

Wczytujemy stworzony wcześniej plik z danymi na jeden miesiąc:

In [9]:
df = pd.read_csv('/content/nasa_dec22_loc.csv')
df

,Year,Month,lon,lat,SWdown,LWdown,SWnet,LWnet,Qle,Qh,...,CCond,RCS,RCT,RCQ,RCSOL,RSmin,RSMacr,LAI,GVEG,Streamflow
0,2022,12,-124.9375,48.8125,40.72300,284.6229,33.27299,-34.09100,7.621784,-6.194625,...,0.000158,0.192282,0.266507,0.982483,0.178051,300.0,0.0,5.032178,0.396436,NaN
1,2022,12,-124.9375,48.9375,42.62393,285.9455,34.81318,-33.75555,8.208872,-4.633200,...,0.000204,0.194993,0.274185,0.983927,0.214745,300.0,0.0,5.032178,0.396436,NaN
2,2022,12,-124.9375,49.0625,40.94436,271.6205,22.50860,-32.55861,4.419356,-7.792607,...,0.000066,0.191651,0.135155,0.986417,0.144038,300.0,0.0,5.037270,0.399030,NaN
3,2022,12,-124.9375,49.1875,42.99100,270.4399,17.68588,-28.97580,2.916802,-7.899727,...,0.000068,0.194352,0.131454,0.986134,0.151408,300.0,0.0,5.042451,0.372922,NaN
4,2022,12,-124.9375,49.3125,42.13938,283.4515,33.73494,-32.91748,5.204706,-0.765860,...,0.000200,0.192913,0.257460,0.983836,0.225124,300.0,0.0,5.046806,0.346801,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76058,2022,12,-67.0625,52.4375,35.15404,235.0424,14.51451,-23.75914,6.920296,-11.013980,...,0.000000,0.114930,0.000000,0.992321,0.004301,70.0,0.0,3.374073,0.003851,NaN
76059,2022,12,-67.0625,52.5625,34.15685,234.6051,13.88118,-23.25308,6.778492,-11.332800,...,0.000000,0.113039,0.000000,0.992282,0.001531,70.0,0.0,3.374073,0.003851,NaN
76060,2022,12,-67.0625,52.6875,33.53803,235.3438,13.45282,-22.54650,6.452690,-10.749050,...,0.000000,0.111853,0.000000,0.992178,0.000570,70.0,0.0,3.374073,0.003851,NaN
76061,2022,12,-67.0625,52.8125,33.70605,234.0825,13.36102,-22.28226,5.570837,-9.454966,...,0.000000,0.112245,0.000000,0.992399,0.000138,70.0,0.0,3.369360,0.002503,NaN


Definiujemy funkcję, która umożliwi tworzenie własnych zakresów kolorów użytych do map:

In [22]:
'''
Funkcja jako argumenty bierze listę wartości określających granice przedziałów liczbowych, które
będą określać jak dla rozważanego parametru mają zmieniać się kolory punktów, których lista stanowi
drugi argument funkcji.
'''
def get_colormap(values, colors_palette, name = 'custom'):
    values = np.sort(np.array(values))
    values = np.interp(values, (values.min(), values.max()), (0, 1))
    cmap = mpl.colors.LinearSegmentedColormap.from_list(name, list(zip(values, colors_palette)))
    return cmap

Tworzymy pierwszą mapę, w tym wypadku dla parametru GVEG związanego z poziomem roślinności:

In [25]:
colormap_GVEG = get_colormap([0, max(df.GVEG.values)], ['yellow', 'darkgreen'])

gdf_GVEG = gv.Points(df, ['lon', 'lat'], ['GVEG']) # obiekt zawierający punkty

tiles_GVEG = gts.OSM # wybieramy mapę tła, w tym wypadku OpenStreetMap

# łączymy mapę tła z punktami i ustawiamy wybrane parametry wizualizacji
map_with_points_gts_GVEG = tiles_GVEG * gdf_GVEG.opts(
    color = 'GVEG',
    cmap = colormap_GVEG,
    size = 8,
    width = 900,
    height = 600,
    colorbar = True,
    toolbar = 'above',
    tools = ['hover', 'wheel_zoom', 'reset'],
    alpha = 0.5 # przezroczystość
    )

map_with_points_gts_GVEG.opts(bgcolor='white')

# zapisujemy mapę do pliku .html
hv.save(map_with_points_gts_GVEG.opts(width=900, height=600, bgcolor='white', xaxis=None, yaxis=None), 'output_map_osm_GVEG.html')
#hv.save(map_with_points_gts_GVEG.opts(width=900, height=600, xaxis=None, yaxis=None), 'output_map_osm_GVEG_3.html')

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)


Druga mapa, tym razem dla parametru Rainf określającego poziom opadów deszczu:

In [45]:
colormap_Rainf = get_colormap([0, max(df.Rainf.values)/2, max(df.Rainf.values)], ['white', 'blue', 'black'])

gdf_Rainf = gv.Points(df, ['lon', 'lat'], ['Rainf'])

tiles_Rainf = gts.OSM

map_with_points_gts_Rainf = tiles_Rainf * gdf_Rainf.opts(
    color = 'Rainf',
    cmap = colormap_Rainf,
    size = 5,
    width = 900,
    height = 600,
    colorbar = True,
    toolbar = 'above',
    tools = ['hover', 'wheel_zoom', 'reset'],
#    alpha = 0.75
    )

map_with_points_gts_Rainf.opts() #bgcolor='black')

# hv.save(map_with_points_gts_Rainf.opts(width=900, height=600, bgcolor='black', xaxis=None, yaxis=None), 'output_map_osm_Rainf.html')
hv.save(map_with_points_gts_Rainf.opts(width=900, height=600, xaxis=None, yaxis=None), 'output_map_osm_Rainf.html')

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)


Trzecia mapa dla AvgSurfT, tj. średniej temperatury powierzchni:

In [44]:
colormap_AvgSurfT = 'cet_rainbow_bgyr_10_90_c83'

gdf_AvgSurfT = gv.Points(df, ['lon', 'lat'], ['AvgSurfT'])

tiles_AvgSurfT = gts.OSM

map_with_points_gts_AvgSurfT = tiles_AvgSurfT * gdf_AvgSurfT.opts(
    color = 'AvgSurfT',
    cmap = colormap_AvgSurfT
    size = 5,
    width = 900,
    height = 600,
    colorbar = True,
    toolbar = 'above',
    tools = ['hover', 'wheel_zoom', 'reset'],
    alpha = 0.7 # przezroczystość
    )

map_with_points_gts_AvgSurfT.opts(bgcolor='white')

hv.save(map_with_points_gts_AvgSurfT.opts(width=900, height=600, bgcolor='white', xaxis=None, yaxis=None), 'output_map_osm_AvgSurfT.html')

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
